In [1]:
import os, sys
from pathlib import Path
import cortex
import nibabel as nib
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as pl
import time
import platform
import subprocess

In [2]:
data_dir = Path("/tank/shared/2024/visual/AOT/derivatives")

subj_nr = 2
sub = 'sub-{0:03d}'.format(subj_nr)
cx_sub = "AOT_" + sub 

fs_dir = data_dir/ "fmripreps/aotfull_preprocs/fullpreproc1/sourcedata/freesurfer"
fs_subj_dir = Path(fs_dir, sub)

glm_dir = data_dir / 'glmsingle'     
# glm_subj_dir = glm_dir / sub
# glm_subj_gfeat_dir = glm_subj_dir / 'func' / 'sub-01_task-ER_space-T1w_desc-preproc_bold.gfeat'

ctx_dir = data_dir / 'cortex' / 'outputs'

In [3]:
print('cortex.database.default_filestore: {}'.format(cortex.database.default_filestore))
print('cortex.options.usercfg: {}'.format(cortex.options.usercfg))

cortex.database.default_filestore: /tank/zhangs/locallib/pycortex/cortex/../filestore/db
cortex.options.usercfg: /tank/zhangs/.config/pycortex/options.cfg


In [4]:

cortex.freesurfer.import_subj(freesurfer_subject=sub,
                                pycortex_subject=cx_sub,
                                freesurfer_subject_dir=fs_dir)

#cortex.freesurfer.import_subj?



b'created by heij on Thu Jul  6 14:30:42 2023\n'
b'created by heij on Thu Jul  6 15:47:40 2023\n'
b'created by heij on Thu Jul  6 14:30:42 2023\n'
b'created by heij on Thu Jul  6 16:40:28 2023\n'


In [5]:
!ls /tank/shared/2024/visual/AOT/derivatives/fmripreps/aotfull_preprocs/fullpreproc1/sourcedata/freesurfer/sub-01/mri/

ls: cannot access '/tank/shared/2024/visual/AOT/derivatives/fmripreps/aotfull_preprocs/fullpreproc1/sourcedata/freesurfer/sub-01/mri/': No such file or directory


In [6]:
example_R2 = "/tank/shared/2024/visual/AOT/derivatives/glmsingle/mainexp/sub-002_ses-01_T1W_nordicstc_mainfull/niiaftersync/R2_syns.nii.gz"
bref = nib.load(example_R2)
transform = cortex.xfm.Transform(np.identity(4), bref)
transform.save(cx_sub, 'fmriprep', 'magnet')

In [7]:
example_anat_fn = (
    fs_subj_dir / "mri" / "T2.mgz"
)  # /tank/shared/2024/visual/AOT/derivatives/fmripreps/aotfull_preprocs/fullpreproc1/sourcedata/freesurfer/sub-002/mri/T2.mgz
print(example_anat_fn)
aref = nib.load(example_anat_fn)
transform = cortex.xfm.Transform(np.identity(4), aref)
transform.save(cx_sub, 'anat', 'magnet')

/tank/shared/2024/visual/AOT/derivatives/fmripreps/aotfull_preprocs/fullpreproc1/sourcedata/freesurfer/sub-002/mri/T2.mgz


In [10]:
mean_func_vcx = cortex.Volume(
    str(example_R2), cx_sub, "fmriprep", cmap="bone", vmin=0, vmax=16000
)
T1_anat_vcx = cortex.Volume(str(fs_subj_dir / 'mri' / 'T2.mgz'), cx_sub, 'anat', cmap='Greens', vmin=0, vmax=1)

ds = {"R2": mean_func_vcx, "myelin": T1_anat_vcx}
'''
for cope, name in zip((1,2,3,4), ["positive", "negative", "pos-neg", "neg-pos"]):
    fracvalue = "/tank/shared/2024/visual/AOT/derivatives/glmsingle/mainexp/sub-002_ses-01_T1W_nordicstc_mainfull/TYPED_FITHRF_GLMDENOISE_RR/FRACvalue.nii"
    cope_vcx = cortex.Volume(str(fracvalue), cx_sub, 'fmriprep', cmap='coolwarm', vmin=-4.5, vmax=4.5)
    ds[name] = cope_vcx
'''

# cortex.webgl.show(ds)
cortex.webgl.make_static(ctx_dir / sub, ds)